# Use Spark to predict product line with `ibm-watson-machine-learning`

This notebook contains steps and code to get data from the IBM Data Science Experience Community, create a predictive model, and start scoring new data. It introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.6 and Apache® Spark 2.4.

You will use a publicly available data set, **GoSales Transactions**, which details anonymous outdoor equipment purchases. Use the details of this data set to predict clients' interests in terms of product line, such as golf accessories, camping equipment, and others.

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Deploy a model for online scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.
-  Explore and visualize prediction result using the plotly package.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#persistence)
5.	[Predict locally](#visualization)
6.	[Deploy and score](#scoring)
7.  [Clean up](#cleanup)
8.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `password`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
password = 'PASTE YOUR PASSWORD HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [2]:
wml_credentials = {
    "username": username,
    "password": password,
    "url": url,
    "instance_id": 'openshift',
    "version": '3.5'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [3]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd3.5/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [4]:
client.set.default_space(space_id)

'SUCCESS'

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then *read* method. 

### Test Spark

In [5]:
try:
    from pyspark.sql import SparkSession
except:
    print('Error: Spark runtime is missing. If you are using Watson Studio change the notebook runtime to Spark.')
    raise

The csv file GoSales_Tx.csv is availble on the same repository where this notebook is located. Load the file to Apache® Spark DataFrame using code below.

In [6]:
import os
from wget import download

sample_dir = 'spark_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename = os.path.join(sample_dir, 'GoSales_Tx.csv')
if not os.path.isfile(filename):
    filename = download('https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd3.5/data/product-line-prediction/GoSales_Tx.csv', out=sample_dir)

In [7]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)
df_data.take(3)

[Row(GENDER='M', AGE=27, MARITAL_STATUS='Single', PROFESSION='Professional', PRODUCT_LINE='Personal Accessories'),
 Row(GENDER='F', AGE=39, MARITAL_STATUS='Single', PROFESSION='Executive', PRODUCT_LINE='Personal Accessories'),
 Row(GENDER='M', AGE=39, MARITAL_STATUS='Married', PROFESSION='Student', PRODUCT_LINE='Mountaineering Equipment')]

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [8]:
df_data.printSchema()

root
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)
 |-- PRODUCT_LINE: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [9]:
df_data.show()

+------+---+--------------+------------+--------------------+
|GENDER|AGE|MARITAL_STATUS|  PROFESSION|        PRODUCT_LINE|
+------+---+--------------+------------+--------------------+
|     M| 27|        Single|Professional|Personal Accessories|
|     F| 39|        Single|   Executive|Personal Accessories|
|     M| 39|       Married|     Student|Mountaineering Eq...|
|     F| 56|        Single| Hospitality|Personal Accessories|
|     M| 45|       Married|     Retired|      Golf Equipment|
|     M| 45|       Married|     Retired|      Golf Equipment|
|     M| 39|       Married|     Student|   Camping Equipment|
|     M| 49|       Married|     Student|   Camping Equipment|
|     F| 49|       Married|     Retired|  Outdoor Protection|
|     M| 47|       Married|     Retired|      Golf Equipment|
|     M| 47|       Married|     Retired|      Golf Equipment|
|     M| 21|        Single|      Retail|Mountaineering Eq...|
|     F| 66|        Single|   Executive|Personal Accessories|
|     M|

In [10]:
df_data.count()

60252

As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [11]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

Number of training records: 48176
Number of testing records : 10860
Number of prediction records : 1216


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [12]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [13]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [14]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [15]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [16]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [17]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [18]:
train_data.printSchema()

root
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)
 |-- PRODUCT_LINE: string (nullable = true)



In [19]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [20]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.672376
Test Error = 0.327624


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using python client libraries.

**Note**: Apache® Spark 2.4 is required.

### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [21]:
saved_model = client.repository.store_model(
    model=model_rf, 
    meta_props={
        client.repository.ModelMetaNames.NAME:'Product Line model',
        client.repository.ModelMetaNames.TYPE: "mllib_2.4",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_id_by_name('spark-mllib_2.4'),
        client.repository.ModelMetaNames.LABEL_FIELD: "PRODUCT_LINE",
    },  
    training_data=train_data, 
    pipeline=pipeline_rf)

Get saved model metadata from Watson Machine Learning.

In [22]:
published_model_id = client.repository.get_model_uid(saved_model)

print("Model Id: " + str(published_model_id))

Model Id: 04cff12b-ff02-43a8-ac79-1a800fe3b9cb


**Model Id** can be used to retrive latest model version from Watson Machine Learning instance.

Below you can see stored model details.

In [24]:
client.repository.get_model_details(published_model_id)

{'entity': {'label_column': 'PRODUCT_LINE',
  'pipeline': {'id': 'f2cb6ba3-b1d8-4722-9062-774dc6f7ea65'},
  'software_spec': {'id': '390d21f8-e58b-4fac-9c55-d7ceda621326',
   'name': 'spark-mllib_2.4'},
  'training_data_references': [{'connection': {'access_key_id': 'not_applicable',
     'endpoint_url': 'not_applicable',
     'secret_access_key': 'not_applicable'},
    'id': '1',
    'location': {},
    'schema': {'fields': [{'metadata': {},
       'name': 'GENDER',
       'nullable': True,
       'type': 'string'},
      {'metadata': {}, 'name': 'AGE', 'nullable': True, 'type': 'integer'},
      {'metadata': {},
       'name': 'MARITAL_STATUS',
       'nullable': True,
       'type': 'string'},
      {'metadata': {},
       'name': 'PROFESSION',
       'nullable': True,
       'type': 'string'},
      {'metadata': {'modeling_role': 'target'},
       'name': 'PRODUCT_LINE',
       'nullable': True,
       'type': 'string'}],
     'id': '1',
     'type': 'struct'},
    'type': 'fs'}],


### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [25]:
loaded_model = client.repository.load(published_model_id)

In [26]:
print(type(loaded_model))

<class 'pyspark.ml.pipeline.PipelineModel'>


As you can see the name is correct. You have already learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally

In this section you will learn how to score test data using loaded model and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [27]:
predictions = loaded_model.transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [28]:
predictions.show(5)

+------+---+--------------+------------+--------------------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|GENDER|AGE|MARITAL_STATUS|  PROFESSION|        PRODUCT_LINE|label|PROFESSION_IX|GENDER_IX|MARITAL_STATUS_IX|          features|       rawPrediction|         probability|prediction|      predictedLabel|
+------+---+--------------+------------+--------------------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|     F| 19|        Single|   Executive|Personal Accessories|  1.0|          2.0|      1.0|              1.0|[1.0,19.0,1.0,2.0]|[0.81953057650034...|[0.04097652882501...|       1.0|Personal Accessories|
|     F| 19|        Single|   Executive|Personal Accessories|  1.0|          2.0|      1.0|              1.0|[1.0,19.0,1.0,2.0]|[0.81953057650034...|[0.04097652882501...|       1.0|Persona

By tabulating a count, you can see which product line is the most popular.

In [29]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show()

+--------------------+-----+
|      predictedLabel|count|
+--------------------+-----+
|   Camping Equipment|  511|
|      Golf Equipment|  152|
|  Outdoor Protection|   39|
|Mountaineering Eq...|  229|
|Personal Accessories|  285|
+--------------------+-----+



<a id="scoring"></a>
## 6. Deploy and score

In this section you will learn how to create online scoring and to score a new data record using `ibm-watson-machine-learning`.

**Note:** You can also use REST API to deploy and score.
For more information about REST APIs, see the [Swagger Documentation](https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_create).

### 6.1: Create online scoring endpoint

Now you can create an online scoring endpoint. 

#### Create online deployment for published model

In [30]:
deployment_details = client.deployments.create(
    published_model_id, 
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "Product Line model deployment",
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)



#######################################################################################

Synchronous deployment creation for uid: '04cff12b-ff02-43a8-ac79-1a800fe3b9cb' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='dfe9314f-cac8-443a-95b3-1d9242c20a86'
------------------------------------------------------------------------------------------------




In [31]:
deployment_details

{'entity': {'asset': {'id': '04cff12b-ff02-43a8-ac79-1a800fe3b9cb'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'S', 'num_nodes': 1},
  'name': 'Product Line model deployment',
  'online': {},
  'space_id': '83b00166-9047-4159-b777-83dcb498e7ab',
  'status': {'online_url': {'url': 'https://wmlgmc-cpd-wmlgmc.apps.wmlautoai.cp.fyre.ibm.com/ml/v4/deployments/dfe9314f-cac8-443a-95b3-1d9242c20a86/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2020-12-08T12:18:04.959Z',
  'id': 'dfe9314f-cac8-443a-95b3-1d9242c20a86',
  'modified_at': '2020-12-08T12:18:04.959Z',
  'name': 'Product Line model deployment',
  'owner': '1000330999',
  'space_id': '83b00166-9047-4159-b777-83dcb498e7ab'}}

Now, you can use above scoring url to make requests from your external application.

<a id="cleanup"></a>
## 7. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd3.5/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 8. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. 
 
 Check out our [Online Documentation](https://dataplatform.cloudibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM

Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.